<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [ ]:
# %pip install -Uqqq rich openai==0.27.2 tiktoken wandb tenacity
# %pip install weave

In [3]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog
import weave

In [4]:
# Download files on colab
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/{examples,prompt_template,system_template}.txt

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [5]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [7]:
# start logging to W&B
# autolog({"project":"llmapps", "job_type": "generation"})
weave.init(project_name="llmapps", settings="Generation")

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
weave: Logged in as Weights & Biases user: nazalint.
weave: View Weave data at https://wandb.ai/My1Team/llmapps/weave


weave: retry_attempt


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [12]:
client = openai.OpenAI()
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [13]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [ ]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

@weave.op()
def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

Sure, here is a sample support question from a user regarding weight and balance calculations: "I'm having trouble 
entering the correct fuel weight in the W&B tool. Can you provide guidance on how to accurately input the fuel     
weight for my aircraft?"

Sure, here is an example of a support question that a user might have about Weights & Biases:                      

"How can I compare the performances of different models using Weights & Biases?"

Sure, here is a support question that a user may have regarding weights and biases:                                

"How can I visualize multiple runs in a single graph using W&B, to compare different training settings of my       
machine learning model?"

Sure! Here's a support question that a W&B user might ask:                                                         

"I'm having trouble understanding how to interpret the graphs and visualizations in my W&B project. Can you provide
some guidance on how to analyze and derive insights from these visualizations effectively?"

"Is there a way to adjust the date range for the insights generated in the Workbench dashboard?"

weave: retry_attempt
weave: retry_attempt


# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [15]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [16]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "can we edit table after run?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [17]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: After my model
finished training, wandb stayed at this point for up to 20 minutes {wandb: / 0.063 MB of 0.063 MB uploaded (0.000  
MB deduped)}. Does anyone know how to solve this? I wrote four pieces of code for training at the same time, and   
they used the same wandb.init method. Is it affected by this? Because when I tested the code before, there was no  
situation where it stayed for 20 minutes. This is my init code:                                                    
wandb.init(project="BioRED_cu05_1",name=args.wandbName) s there a good way to use sweeps and launch? Is there a way
to download a public W&B artifact without the requirement of W&B API key ? Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [18]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

How can I analyze the results from multiple runs in a sweep to compare the performance of different configurations 
effectively?

"What could be causing my W&B to stay at the upload point for up to 20 minutes after model training completes? I   
have multiple pieces of code running training simultaneously using the same wandb.init method. Could this be a     
factor? Here is my init code: wandb.init(project="BioRED_cu05_1", name=args.wandbName)."

"Why is my W&B dashboard showing that my model has been training for a long time after it has finished training    
successfully? I used the same wandb.init() method for different pieces of training code simultaneously. Could this 
be causing the delay? Here is my init code: wandb.init(project="BioRED_cu05_1", name=args.wandbName)."

How can I resolve the issue of my model training finishing but WandB being stuck at a particular point for up to 20
minutes? My code includes multiple training processes using the same wandb.init method, so could this be impacting 
the duration of data upload to WandB?

How can I troubleshoot and resolve the issue of wandb staying at a certain point for up to 20 minutes after my     
model finishes training?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [20]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

In [21]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [22]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[537, 2529, 4179, 803, 2596, 1206, 1644, 956, 2093, 2940, 365]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [23]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [24]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [25]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B     
   documentation. Below you will find a fragment of W&B documentation: W&B Tables with this 5 minute Quickstart.   

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                Tables Quickstart                                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The following Quickstart demonstrates how to log data tables, visualize data, and query data.                      

🌆 ]8;id=144457;http://wandb.me/tables-quickstart\Open In Colab]8;;\ Select the button below to try a PyTorch Quickstart example project on MNIST data.                                 


                                                  1. Log a table                                                   

Follow the procedure outlined below to log a Table with W&B:                                                       

 1 Initialize a W&B Run with ]8;id=979346;../../ref/python/init.md\wandb.init()]8;;\.                                                                         
 2 Create a ]8;id=572502;../../ref/python/data-types/table.md\wandb.Table()]8;;\ object instance. Pass the name of the columns in your table along with the data for the  
   columns and data parameters, respectively.                                                                      
 3 Log the table with ]8;id=619556;../../ref/python/log.md\run.log()]8;;\ as a key-value pair. Provide a name for your table for the key, and pass the object
   instance of wandb.Table as the value.                                                                           

                                                                                                                   
 run = wandb.init(project="table-test")                                                                            
 my_table = wandb.Table(columns=["a", "b"], data=[["a1", "b1"], ["a2", "b2"]])                                     
 run.log({"Table Name": my_table})                                                                                 
                                                                                                                   

You can optionally pass in a Pandas DataFrame to wandb.Table() Class. For more information on supported data types,
see the ]8;id=328602;../../ref/python/data-types/table.md\wandb.Table]8;;\ in the W&B API Reference Guide.                                                                


                                       2. Visualize tables in the workspace                                        

View the resulting table in your workspace. Navigate to the W&B App and select the name of your Run in your Project
workspace. A new panel is added for each unique table key.                                                         

🌆 ]8;id=96631;/images/data_vis/wandb_demo_logged_sample_table.png\wandb_demo_logged_sample_table.png]8;;\                                                                                                                    

In this example, my_table, is logged under the key "Table Name".                                                   


                                         3. Compare across model versions                                          

Log sample tables from multiple W&B Runs and compare results in the project workspace. In this ]8;id=333199;https://wandb.ai/carey/table-test?workspace=user-carey\example workspace]8;;\,  
we show how to combine rows from multiple different versions in the same table.                                    

🌆 ]8;id=829458;/images/data_vis/wandb_demo_toggle_on_and_off_cross_run_comparisons_in_tables.gif\wandb_demo_toggle_on_and_off_cross_run_comparisons_in_

Let's generate 3 possible questions:

In [26]:
generate_and_print(system_prompt, generation_prompt, n=3)

Support Question: How can I log a table with the W&B API?                                                          

Answer: You can log a table with the W&B API by following these steps:                                             

 1 Initialize a W&B Run with wandb.init().                                                                         
 2 Create a wandb.Table() object instance, specifying the columns and data for the table.                          
 3 Log the table using run.log() by providing a key (table name) and the wandb.Table object instance as the value. 

For example:                                                                                                       

                                                                                                                   
 import wandb                                                                                                      
                                                                                                                   
 run = wandb.init(project="table-test")                                                                            
 my_table = wandb.Table(columns=["a", "b"], data=[["a1", "b1"], ["a2", "b2"]])                                     
 run.log({"Table Name": my_table})                                                                                 
                                                                                                                   

You can also pass a Pandas DataFrame to wandb.Table() if needed. For more information on supported data types,     
refer to the ]8;id=683441;../../ref/python/data-types/table.md\wandb.Table]8;;\]8;id=188530;../../ref/python/data-types/table.md\ documentation]8;;\.

Support question: How can I log a table with W&B using a Pandas DataFrame?                                         

Answer: You can log a table with W&B using a Pandas DataFrame by initializing a W&B Run, creating a wandb.Table()  
object instance, and passing the DataFrame to the wandb.Table() Class. You can then log the table with run.log() as
a key-value pair. For more information, you can refer to the ]8;id=821034;../../ref/python/data-types/table.md\wandb.Table]8;;\ documentation in the W&B API Reference    
Guide.

Support Question: How can I log a table with W&B and visualize it in the workspace?                                

Answer: To log a table with W&B, you can follow these steps:                                                       

 1 Initialize a W&B Run using wandb.init().                                                                        
 2 Create a wandb.Table() object instance by specifying the columns and data for the table.                        
 3 Log the table using run.log() by providing a name for your table and passing the wandb.Table object instance as 
   the value.                                                                                                      

You can then visualize the table in your workspace by navigating to the W&B App, selecting the name of your Run,   
and viewing the table under the corresponding key.

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [27]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [28]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [29]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [30]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [31]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [32]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [33]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
how can i get all the versions of an aritfact of a particular type? i have my log from training in a txt file. Can 
i import that data to wandb so i can visualize the data? Explain how I can version datasets with Weights & Biases. 
How do I create a new version of a dataset? How does Weights & Biases work with versions? What are some best       
practices for versioning datasets?                                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                     W&B Tables with this 5 minute Quickstart.                                     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                Tables Quickstart                                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The following Quickstart demonstrates how to log data tables, visualize data, and query data.                      

🌆 ]8;id=387630;http://wandb.me/tables-quickstart\Open In Colab]8;;\ Select the button below to try a PyTorch Quickstart example project on MNIST data.                                 


                                                  1. Log a table                                                   

Follow the procedure outlined below to log a Table with W&B:                                                       

 1 Initialize a W&B Run with ]8;id=409916;../../ref/python/init.md\wandb.init()]8;;\.                                                                         
 2 Create a ]8;id=835633;../../ref/python/data-types/table.md\wandb.Table()]8;;\ object instance. Pass the name of the columns in your table along with the data for the  
   columns and data parameters, respectively.                                                                      
 3 Log the table with ]8;id=21264;../../ref/python/log.md\run.log()]8;;\ as a key-value pair. Provide a name for your table for the key, and pass the object
   instance of wandb.Table as the value.                                                                           

                                                                                                                   
 run = wandb.init(project="table-test")                                                                            
 my_table = wandb.Table(columns=["a", "b"], data=[["a1", "b1"], ["a2", "b2"]])                                     
 run.log({"Table Name": my_table})                                                                                 
                                                                                                                   

You can optionally pass in a Pandas DataFrame to wandb.Table() Class. For more information on supported data types,
see the ]8;id=274527;../../ref/python/data-types/table.md\wandb.Table]8;;\ in the W&B API Reference Guide.                                                                


                                       2. Visualize tables in the workspace                                        

View the resulting table in your workspace. Navigate to the W&B App and select the name of your Run in you

In [34]:
@weave.op()
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [35]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [36]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

("\nThe user is trying to log and manipulate data tables using W&B. They have a Pandas DataFrame and they aren't sure about how to log it as a table. \n",
 '\nIs it possible to directly log a Pandas DataFrame as a Wandb Table? If so, how do I go about it?\n',
 '\nYes, you can directly log a Pandas DataFrame as a Wandb Table. Here\'s how:\n\n1. Initialize a W&B Run using `wandb.init()` function.\n2. Create a `wandb.Table()` instance and pass your DataFrame to it.\n\nBelow is a Python code example:\n```python\nrun = wandb.init(project="table-test")\nmy_df = pd.DataFrame(data=[["a1", "b1"], ["a2", "b2"]], columns=["a", "b"])\nmy_table = wandb.Table(dataframe=my_df)\nrun.log({"Table Name": my_table})\n```\n\nThis logs `my_df`, your Pandas DataFrame as a Wandb Table, which can then be visualized in your W&B workspace. For more information on supported data types, refer to the `wandb.Table` in the W&B API Reference Guide.')

weave: retry_attempt


In [38]:
wandb.init(project='llmapps', job_type="Generation")

wandb: Currently logged in as: nazalint (My1Team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [39]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

<Artifact generated_examples>

In [ ]:
# wandb.finish()